# Hospital billing dataset preparation

In [ ]:
# Skip this cell, at first
import os
os.chdir('../..')
os.getcwd()

In [1]:
import pandas as pd
import numpy as np
from CI_Experiments.config import PROJECT_DIR # # if this line results ModuleNotFoundError, then execute the cell above

In [2]:
UNPREPARED_PATH = PROJECT_DIR / 'data/unprepared_process_logs'
HOSPITAL_BILLING_DATA = UNPREPARED_PATH / 'hospital_billing.csv'
DESTINATION_DATA = PROJECT_DIR / 'data/prepared_process_logs/hospital_billing/activity/activity.csv'

In [3]:
unprepared_data = pd.read_csv(HOSPITAL_BILLING_DATA, sep=';')
unprepared_data.head()

,speciality,Case ID,Activity,Resource,actOrange,actRed,blocked,caseType,diagnosis,flagC,...,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
0,C,RB,NEW,ResN,missing,missing,False,B,missing,missing,...,2012-12-13 11:13:18,673,12,3,11,0.0,0.0,1,0,regular
1,I,R,NEW,other,missing,missing,False,A,S,missing,...,2012-12-13 12:12:05,732,12,3,12,0.0,0.0,1,1,regular
2,M,PB,NEW,ResNA,missing,missing,False,B,missing,missing,...,2012-12-13 12:18:50,738,12,3,12,0.0,0.0,1,2,deviant
3,M,PB,DELETE,ResNA,missing,missing,False,B,missing,missing,...,2012-12-13 15:06:26,906,12,3,15,167.6,167.6,2,3,deviant
4,C,ND,NEW,ResUB,missing,missing,False,B,missing,missing,...,2012-12-13 16:30:12,990,12,3,16,0.0,0.0,1,2,regular


In [4]:
unprepared_data.columns

Index(['speciality', 'Case ID', 'Activity', 'Resource', 'actOrange', 'actRed',
       'blocked', 'caseType', 'diagnosis', 'flagC', 'flagD', 'msgCode',
       'msgType', 'state', 'version', 'isCancelled', 'closeCode', 'msgCount',
       'Complete Timestamp', 'timesincemidnight', 'month', 'weekday', 'hour',
       'timesincelastevent', 'timesincecasestart', 'event_nr', 'open_cases',
       'label'],
      dtype='object')

In [5]:
four_columns_for_activity = [
    'Case ID',
    'label',
    'Activity',
    'Complete Timestamp'
]

In [6]:
activity_data = unprepared_data[four_columns_for_activity]
activity_data.head()

,Case ID,label,Activity,Complete Timestamp
0,RB,regular,NEW,2012-12-13 11:13:18
1,R,regular,NEW,2012-12-13 12:12:05
2,PB,deviant,NEW,2012-12-13 12:18:50
3,PB,deviant,DELETE,2012-12-13 15:06:26
4,ND,regular,NEW,2012-12-13 16:30:12


In [7]:
activity_data['label'] = np.where(activity_data['label'] == 'deviant', 1, 0)
activity_data.head()

/var/folders/2v/0cy8xvx12vl1y6x611k263_w0000gn/T/ipykernel_84401/2959549615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_data['label'] = np.where(activity_data['label'] == 'deviant', 1, 0)


,Case ID,label,Activity,Complete Timestamp
0,RB,0,NEW,2012-12-13 11:13:18
1,R,0,NEW,2012-12-13 12:12:05
2,PB,1,NEW,2012-12-13 12:18:50
3,PB,1,DELETE,2012-12-13 15:06:26
4,ND,0,NEW,2012-12-13 16:30:12


In [8]:
activity_data = activity_data.rename(columns={'label': 'Outcome', 'Complete Timestamp': 'Timestamp'})
activity_data.head()

,Case ID,Outcome,Activity,Timestamp
0,RB,0,NEW,2012-12-13 11:13:18
1,R,0,NEW,2012-12-13 12:12:05
2,PB,1,NEW,2012-12-13 12:18:50
3,PB,1,DELETE,2012-12-13 15:06:26
4,ND,0,NEW,2012-12-13 16:30:12


In [13]:
activity_data.to_csv(DESTINATION_DATA, index=False)

In [9]:
activity_data['Activity'].unique()

array(['NEW', 'DELETE', 'CHANGE DIAGN', 'FIN', 'RELEASE', 'CODE OK',
       'REOPEN', 'BILLED', 'SET STATUS', 'STORNO', 'REJECT', 'JOIN-PAT',
       'CHANGE END', 'CODE NOK', 'CODE ERROR', 'MANUAL', 'other', 'EMPTY'],
      dtype=object)

In [10]:
np.mean(activity_data.groupby('Case ID').count()['Activity'])

5.528874556594647